<a href="https://colab.research.google.com/github/derzhavin3016/CompMath/blob/master/Lab11/Lab11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Домашняя лабораторная работа №11 по вычислительной математике

Державин Андрей, Б01-909

## Задача 2

Дифференциальная задача
$$
\left\lbrace
\begin{matrix}
\dfrac{\partial u}{\partial t} &=& \dfrac{\partial}{\partial x} \left(u^2 \dfrac{\partial u}{\partial x} \right), & 0 < t \leq 1, 0 < x < 1 \\
u(0, x) &=& \dfrac{1+x}{\sqrt{5}}, & 0 \leq x \leq 1 \\
u(t, 0) &=& \dfrac{1}{\sqrt{5 - 4t}}, & 0 < t \leq 1 \\
u(t, 1) &=& \dfrac{2}{\sqrt{5 - 4t}}, & 0 < t \leq 1 \\
\end{matrix}
\right.
$$

In [ ]:
import numpy as np
from matplotlib import pyplot as plt